In [47]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt

# Retriving Dataset

## JHU Dataset

In [48]:
df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
df_covid19 = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv")

In [49]:
dates = df_confirmed.columns.values.tolist()[4:]
df_confirmed = pd.melt(df_confirmed, id_vars=['Province/State','Country/Region','Lat','Long'], value_vars= dates)
df_deaths = pd.melt(df_deaths, id_vars=['Province/State','Country/Region','Lat','Long'], value_vars= dates)
df_recovered = pd.melt(df_recovered, id_vars=['Province/State','Country/Region','Lat','Long'], value_vars= dates)

In [50]:
df_confirmed = df_confirmed.rename(columns={'Country/Region':'Country_Region','Province/State':'Province_State','variable': 'Date','value':'Confirmed'})
df_deaths = df_deaths.rename(columns={'Country/Region':'Country_Region','Province/State':'Province_State','variable': 'Date','value':'Deaths'})
df_recovered = df_recovered.rename(columns={'Country/Region':'Country_Region','Province/State':'Province_State','variable': 'Date','value':'Recovered'})

In [51]:
df_confirmed

,Province_State,Country_Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
5,NaN,Antigua and Barbuda,17.060800,-61.796400,1/22/20,0
6,NaN,Argentina,-38.416100,-63.616700,1/22/20,0
7,NaN,Armenia,40.069100,45.038200,1/22/20,0
8,Australian Capital Territory,Australia,-35.473500,149.012400,1/22/20,0
9,New South Wales,Australia,-33.868800,151.209300,1/22/20,0


## US State Level data

In [52]:
US_State = pd.DataFrame()
for month in range(1,4):
    for day in range(1,31):
        month = str(month).zfill(2)
        day = str(day).zfill(2)
        try:
            US_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + month + '-' + day +'-2020.csv'
            df_bin = pd.read_csv(US_url)
            df_bin['Date'] = '2020-'+month+'-'+day
            US_State = pd.concat([US_State, df_bin], ignore_index = True)
        except:
            continue

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [53]:
US_State_copy = US_State.copy()

In [54]:
US_State_copy = US_State_copy[(US_State_copy['Country/Region'] == 'US') | (US_State_copy['Country_Region'] == 'US')]
US_State_copy = US_State_copy.reset_index(drop = True)

# Data Cleaning

In [55]:
US_State_copy.head()

,Active,Admin2,Combined_Key,Confirmed,Country/Region,Country_Region,Date,Deaths,FIPS,Last Update,Last_Update,Lat,Latitude,Long_,Longitude,Province/State,Province_State,Recovered
0,NaN,NaN,NaN,1.0,US,NaN,2020-01-22,NaN,NaN,1/22/2020 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
1,NaN,NaN,NaN,1.0,US,NaN,2020-01-23,NaN,NaN,1/23/20 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
2,NaN,NaN,NaN,1.0,US,NaN,2020-01-24,NaN,NaN,1/24/20 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
3,NaN,NaN,NaN,1.0,US,NaN,2020-01-24,NaN,NaN,1/24/20 17:00,NaN,NaN,NaN,NaN,NaN,Chicago,NaN,NaN
4,NaN,NaN,NaN,1.0,US,NaN,2020-01-25,NaN,NaN,1/25/20 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN


In [56]:
US_State_copy.columns

Index(['Active', 'Admin2', 'Combined_Key', 'Confirmed', 'Country/Region',
       'Country_Region', 'Date', 'Deaths', 'FIPS', 'Last Update',
       'Last_Update', 'Lat', 'Latitude', 'Long_', 'Longitude',
       'Province/State', 'Province_State', 'Recovered'],
      dtype='object')

In [57]:
US_State_copy['Active']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24       NaN
25       NaN
26       NaN
27       NaN
28       NaN
29       NaN
        ... 
30139    0.0
30140    0.0
30141    0.0
30142    0.0
30143    0.0
30144    0.0
30145    0.0
30146    0.0
30147    0.0
30148    0.0
30149    0.0
30150    0.0
30151    0.0
30152    0.0
30153    0.0
30154    0.0
30155    0.0
30156    0.0
30157    0.0
30158    0.0
30159    0.0
30160    0.0
30161    0.0
30162    0.0
30163    0.0
30164    0.0
30165    0.0
30166    0.0
30167    0.0
30168    0.0
Name: Active, Length: 30169, dtype: float64

In [58]:
US_State_copy = US_State_copy.drop(['Active','Admin2','Combined_Key','FIPS'],1)

## Country/Region, Province/State, Latitude, Longitude

In [59]:
US_State_copy[US_State_copy['Country_Region'].isnull()]

,Confirmed,Country/Region,Country_Region,Date,Deaths,Last Update,Last_Update,Lat,Latitude,Long_,Longitude,Province/State,Province_State,Recovered
0,1.0,US,NaN,2020-01-22,NaN,1/22/2020 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
1,1.0,US,NaN,2020-01-23,NaN,1/23/20 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
2,1.0,US,NaN,2020-01-24,NaN,1/24/20 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
3,1.0,US,NaN,2020-01-24,NaN,1/24/20 17:00,NaN,NaN,NaN,NaN,NaN,Chicago,NaN,NaN
4,1.0,US,NaN,2020-01-25,NaN,1/25/20 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
5,1.0,US,NaN,2020-01-25,NaN,1/25/20 17:00,NaN,NaN,NaN,NaN,NaN,Illinois,NaN,NaN
6,1.0,US,NaN,2020-01-26,NaN,1/26/20 16:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
7,1.0,US,NaN,2020-01-26,NaN,1/26/20 16:00,NaN,NaN,NaN,NaN,NaN,Illinois,NaN,NaN
8,2.0,US,NaN,2020-01-26,NaN,1/26/20 16:00,NaN,NaN,NaN,NaN,NaN,California,NaN,NaN
9,1.0,US,NaN,2020-01-26,NaN,1/26/20 16:00,NaN,NaN,NaN,NaN,NaN,Arizona,NaN,NaN


In [60]:
US_State_copy['Country_Region'][:1555] = US_State_copy['Country/Region'][:1555]
US_State_copy = US_State_copy.drop('Country/Region',1)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
# The rest nulls are data for one country, so don't have province/state
US_State_copy[US_State_copy['Province_State'].isnull()]

,Confirmed,Country_Region,Date,Deaths,Last Update,Last_Update,Lat,Latitude,Long_,Longitude,Province/State,Province_State,Recovered
0,1.0,US,2020-01-22,NaN,1/22/2020 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
1,1.0,US,2020-01-23,NaN,1/23/20 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
2,1.0,US,2020-01-24,NaN,1/24/20 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
3,1.0,US,2020-01-24,NaN,1/24/20 17:00,NaN,NaN,NaN,NaN,NaN,Chicago,NaN,NaN
4,1.0,US,2020-01-25,NaN,1/25/20 17:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
5,1.0,US,2020-01-25,NaN,1/25/20 17:00,NaN,NaN,NaN,NaN,NaN,Illinois,NaN,NaN
6,1.0,US,2020-01-26,NaN,1/26/20 16:00,NaN,NaN,NaN,NaN,NaN,Washington,NaN,NaN
7,1.0,US,2020-01-26,NaN,1/26/20 16:00,NaN,NaN,NaN,NaN,NaN,Illinois,NaN,NaN
8,2.0,US,2020-01-26,NaN,1/26/20 16:00,NaN,NaN,NaN,NaN,NaN,California,NaN,NaN
9,1.0,US,2020-01-26,NaN,1/26/20 16:00,NaN,NaN,NaN,NaN,NaN,Arizona,NaN,NaN


In [62]:
def combine_same_columns(col1, col2):
    for i in range(len(col1)):
        if type(col1[i]) == str:
            col2[i] = col1[i]

In [63]:
combine_same_columns(US_State_copy['Province/State'],US_State_copy['Province_State'])
combine_same_columns(US_State_copy['Lat'],US_State_copy['Latitude'])
combine_same_columns(US_State_copy['Long_'],US_State_copy['Longitude'])

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [64]:
# Since we don't need Latitude and Longitude in this analysis, we directly delete these two columns
US_State_copy = US_State_copy.drop(['Province/State','Lat','Long_','Latitude','Longitude'], 1)

In [65]:
US_State_copy.isnull().any()

Confirmed         False
Country_Region     True
Date              False
Deaths             True
Last Update        True
Last_Update        True
Province_State    False
Recovered          True
dtype: bool

In [66]:
US_State_copy = US_State_copy[US_State_copy['Country_Region'].isnull() == False]

In [67]:
US_State_copy.isnull().any()

Confirmed         False
Country_Region    False
Date              False
Deaths             True
Last Update        True
Last_Update        True
Province_State    False
Recovered          True
dtype: bool

In [68]:
# Now all Nan we have is for Deaths and recovered, so we can fill using 0
US_State_copy = US_State_copy.fillna(0)

# Country-level Analysis

## Prepare dataset
### China

In [69]:
def agg_China(df,column):    
    agg_China = pd.DataFrame(df[df['Country_Region'] == 'China'].groupby('Date')[column].sum(),columns = [column])
    agg_China = agg_China.reset_index(drop = False)
    df_China = df[df['Country_Region'] == 'China']
    df_China = df_China.merge(agg_China, left_on = 'Date', right_on = 'Date')
    return df_China

In [70]:
df_confirmed_China = agg_China(df_confirmed,'Confirmed')
df_deaths_China = agg_China(df_deaths,'Deaths')
df_recovered_China = agg_China(df_recovered,'Recovered')

In [71]:
df_confirmed_China = df_confirmed_China[['Country_Region','Date','Confirmed_y']]
df_confirmed_China = df_confirmed_China.drop_duplicates()
df_confirmed_China = df_confirmed_China.rename(columns = {'Confirmed_y':'Confirmed'}).reset_index(drop = True)

df_deaths_China = df_deaths_China[['Country_Region','Date','Deaths_y']]
df_deaths_China = df_deaths_China.drop_duplicates()
df_deaths_China = df_deaths_China.rename(columns = {'Deaths_y':'Deaths'}).reset_index(drop = True)

df_recovered_China = df_recovered_China[['Country_Region','Date','Recovered_y']]
df_recovered_China = df_recovered_China.drop_duplicates()
df_recovered_China = df_recovered_China.rename(columns = {'Recovered_y':'Recovered'}).reset_index(drop = True)

In [72]:
df_deaths_China = pd.DataFrame(df_deaths_China['Deaths'], columns = ['Deaths'])
df_recovered_China = pd.DataFrame(df_recovered_China['Recovered'], columns = ['Recovered'])
df_China = df_confirmed_China.merge(df_deaths_China, left_index = True, right_index = True)
df_China = df_China.merge(df_recovered_China, left_index = True, right_index = True)

### US

In [73]:
df_confirmed_US = df_confirmed[df_confirmed['Country_Region'] == 'US']
df_confirmed_US = df_confirmed_US[['Country_Region','Date','Confirmed']].reset_index(drop = True)

df_deaths_US = df_deaths[df_confirmed['Country_Region'] == 'US']
df_deaths_US = df_deaths_US[['Country_Region','Date','Deaths']].reset_index(drop = True)

df_recovered_US = df_recovered[df_recovered['Country_Region'] == 'US']
df_recovered_US = df_recovered_US[['Country_Region','Date','Recovered']].reset_index(drop = True)

In [74]:
df_deaths_US = pd.DataFrame(df_deaths_US['Deaths'], columns = ['Deaths'])
df_recovered_US = pd.DataFrame(df_recovered_US['Recovered'], columns = ['Recovered'])
df_US = df_confirmed_US.merge(df_deaths_US, left_index = True, right_index = True)
df_US = df_US.merge(df_recovered_US, left_index = True, right_index = True)

### Italy

In [75]:
df_confirmed_Italy = df_confirmed[df_confirmed['Country_Region'] == 'Italy']
df_confirmed_Italy = df_confirmed_Italy[['Country_Region','Date','Confirmed']].reset_index(drop = True)

df_deaths_Italy = df_deaths[df_confirmed['Country_Region'] == 'Italy']
df_deaths_Italy = df_deaths_Italy[['Country_Region','Date','Deaths']].reset_index(drop = True)

df_recovered_Italy = df_recovered[df_recovered['Country_Region'] == 'Italy']
df_recovered_Italy = df_recovered_Italy[['Country_Region','Date','Recovered']].reset_index(drop = True)

In [76]:
df_deaths_Italy = pd.DataFrame(df_deaths_Italy['Deaths'], columns = ['Deaths'])
df_recovered_Italy = pd.DataFrame(df_recovered_Italy['Recovered'], columns = ['Recovered'])
df_Italy = df_confirmed_Italy.merge(df_deaths_Italy, left_index = True, right_index = True)
df_Italy = df_Italy.merge(df_recovered_Italy, left_index = True, right_index = True)

In [77]:
# Add the number of new cases for each day
df_China["Confirmed_new"] = df_China["Confirmed"].diff()
df_China["Deaths_new"] = df_China["Deaths"].diff()
df_China["Recovered_new"] = df_China["Recovered"].diff()

df_Italy["Confirmed_new"] = df_Italy["Confirmed"].diff()
df_Italy["Deaths_new"] = df_Italy["Deaths"].diff()
df_Italy["Recovered_new"] = df_Italy["Recovered"].diff()

df_US["Confirmed_new"] = df_US["Confirmed"].diff()
df_US["Deaths_new"] = df_US["Deaths"].diff()
df_US["Recovered_new"] = df_US["Recovered"].diff()

In [78]:
# Add the Incremental ratio for new cases for each day
df_China["Confirmed_Incremental_Rate"] = df_China["Confirmed"].diff()/df_China["Confirmed"]
df_China["Deaths_Incremental_Rate"] = df_China["Deaths"].diff()/df_China["Deaths"]
df_China["Recovered_Incremental_Rate"] = df_China["Recovered"].diff()/df_China["Recovered"]

df_Italy["Confirmed_Incremental_Rate"] = df_Italy["Confirmed"].diff()/df_Italy["Confirmed"]
df_Italy["Deaths_Incremental_Rate"] = df_Italy["Deaths"].diff()/df_Italy["Deaths"]
df_Italy["Recovered_Incremental_Rate"] = df_Italy["Recovered"].diff()/df_Italy["Recovered"]

df_US["Confirmed_Incremental_Rate"] = df_US["Confirmed"].diff()/df_US["Confirmed"]
df_US["Deaths_Incremental_Rate"] = df_US["Deaths"].diff()/df_US["Deaths"]
df_US["Recovered_Incremental_Rate"] = df_US["Recovered"].diff()/df_US["Recovered"]

In [79]:
df_country = pd.concat([df_China,df_US,df_Italy], axis=0) 

In [80]:
df_country.to_csv('df_country.csv')

In [81]:
df_country.head()

,Country_Region,Date,Confirmed,Deaths,Recovered,Confirmed_new,Deaths_new,Recovered_new,Confirmed_Incremental_Rate,Deaths_Incremental_Rate,Recovered_Incremental_Rate
0,China,1/22/20,548,17,28,NaN,NaN,NaN,NaN,NaN,NaN
1,China,1/23/20,643,18,30,95.0,1.0,2.0,0.147745,0.055556,0.066667
2,China,1/24/20,920,26,36,277.0,8.0,6.0,0.301087,0.307692,0.166667
3,China,1/25/20,1406,42,39,486.0,16.0,3.0,0.345661,0.380952,0.076923
4,China,1/26/20,2075,56,49,669.0,14.0,10.0,0.322410,0.250000,0.204082
